In [ ]:
from datascience import *
%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np
import warnings
warnings.simplefilter(action='ignore')

## Lecture 12 ##

# Joins

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

<br><br><br>

**Exercise:** Join drinks and discounts based on Cafe/Location.

In [ ]:
combined=...
combined

<br><br><br>

**Exercise:** Calculate the lowest rate you can get for each drink using the available coupons.
<br>
1. Create a column in the combined table showing discounted price
2. Select only the drinks and the discounted price
3. How to get the minimum price per drink

In [ ]:
discount_frac = 1-combined.column("Coupon % off")/100.0

(
    ...
)

<br><br><br>
**Exercise:** What happens if I run the following? How many rows will it produce? Why?


In [ ]:
drinks.join('Cafe', drinks, 'Cafe')

## Value Annotation

In [ ]:
drinks

In [ ]:
carbs = Table().with_columns(
    'Drink', make_array('Milk Tea', 'Latte',  'Espresso'),
    'Carbs', make_array('High',     'Medium', 'Low'))
drinks.join('Drink', carbs)

## Bike Sharing ##

In [ ]:
trip = Table.read_table('trip.csv')
trip.show(3)

<br><br><br>
**Exercise:** Create pivot table for Start Station and End Station. What value is being shown in each cell?

<br><br><br>
**Exercise:** Verify the output of the pivot table
1. Filter the table according to the start and end station and count entries

<br><br><br>
**Exercise:** Create pivot table showing average duration of the trips between the  Start Station and End Station ?

In [ ]:
help(Table.pivot)

## Distribution of Durations ##

In [ ]:
trip.hist('Duration')

In [ ]:
trip.sort('Duration', descending=True)

<br><br><br>
**Exercise:** Create histogram with bins ranging from 0 to 1800 in increments of 250

In [ ]:
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
commute.num_rows

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

# <br><br><br>
**Exercise:** Create histogram with more fine grained bins

In [ ]:
commute.hist('Duration', bins=50, unit='Second')

## Start and End Stations ##

In [ ]:
# Most common start station

starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
# Numbers of trips between stations

commute.pivot('Start Station', 'End Station')

In [ ]:
# Average durations of trips between stations



## Fastest Trips between Stations ##

How can we find the fastest trip ever between each pair of stations?

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
# How would we calculate the best case scenario time between stations?

shortest = ..
shortest.show(5)

## Discussion question

Find the 5 stations closest to Civic Center BART by minimum trip time.

In [ ]:
from_cc = shortest.where(
    'Start Station', are.containing('Civic Center BART')).sort(
    'Duration min')
from_cc.take(np.arange(5))

In [ ]:
to_cc = shortest.where(
    'End Station', are.containing('Civic Center BART')).sort(
    'Duration min')
to_cc.take(np.arange(5))

In [ ]:
print(from_cc.num_rows,to_cc.num_rows)

## Maps ##

In [ ]:
# Geographical data on the stations
stations = Table.read_table('station.csv').drop(4, 6)
stations

In [ ]:
sf_stations = stations.where('landmark', are.equal_to('San Francisco'))
sf_stations_map_data = (sf_stations
 .select('lat', 'long', 'name')
 .relabeled('name', 'labels')
                       )
sf_stations_map_data.show(3)

In [ ]:
Marker.map_table(sf_stations_map_data)

In [ ]:
Circle.map_table(sf_stations_map_data)

### Discussion question

Map all stations within 4 minutes (minimum ride time) of Civic Center.

In [ ]:
from_cc.show(3)

In [ ]:
sf_stations_map_data.show(3)

1. Filter out from from_cc all the stations that are with in 4 minutes ride from the Civic center
2. Join/Pivot ??
3. Markers.map_table

In [ ]:
close_cc = from_cc.<>
close_markers = (sf_stations_map_data
      <>
Marker.map_table(close_markers)

## Add Colors!

Choose marker colors by the minimum time from Civic Center

In [ ]:
minutes = np.round(from_cc.column("Duration min") / 60)
print(min(minutes), max(minutes))

In [ ]:
colors = Table().with_columns(
    "minutes", np.arange(15),
    "colors",  ["orange", "orange", 
                "lightgreen", "green", "darkgreen", 
                "lightgray", "lightgray", "lightgray", 
                "gray", "gray", "gray", "darkgray",
                "black", "black", "black"])

colors_cc = (from_cc
 .with_column("Minutes", minutes)
 .join("Minutes", colors, "minutes"))

colored_markers = (sf_stations_map_data
      .join('labels', colors_cc, 'End Station')
      .select('lat', 'long', 'labels', 'colors'))
Marker.map_table(colored_markers)

## Example Midterm Questions

For each of the following, write a Python expression that returns the given data:

1. The name of the station where most rentals ended

2. The number of stations for which the average duration of trips ending at that station was more than 10 minutes

3. The number of stations that have more than 500 starts and more than 500 ends.